# Confronto tra Insertion Sort e Counting Sort
## Matteo Lotti
### Marzo-Aprile 2023

In questo notebook ci poniamo l'obiettivo di impostare e effettuare un confronto tra due algoritmi di ordinamento. In particolare, i due algoritmi in questione saranno:

- __Insertion Sort__

- __Counting Sort__

Il notebook sarà sviluppato in questo modo:

1. Nelle celle successive saranno presenti le implementazioni dei due algoritmi seguiti rispettivamente da una breve spiegazione messa in relazione con le evidenze teoriche conosciute.

2. Successivamente sarà presente una cella nella quale sarà implementata la funzione di test, seguita da un'opportuna spiegazione del codice e dei test che saranno eseguiti

3. Saranno effettuati e descritti i dovuti test e i rispettivi risultati

4. Nella chiusura tireremo le somme dei risultati ottenuti dagli esperimenti svolti

## Implementazioni degli algoritmi
### Insertion Sort

In [26]:
def insertion_sort(arr):
    """Insertion sort algorithm.

    Args:
        arr (list): Array to be sorted.

    Returns:
        list: Sorted array.

    Time complexity: O(n^2)
    """
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and key < arr[j]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
    return arr

Nella cella sovrastante possiamo vedere l'implementazione dell'algoritmo di ordinamento __Insertion Sort__. L'algoritmo si basa su un ciclo _for_ iniziale nel quale si inizializza la variabile _key_ con il valore all'indice _i_ dell'array da ordinare. All'interno del _for_ un ciclo _while_ ci permette di iterare sugli elementi precedenti a _key_ in modo tale da posizionare il valore di _key_ nella posizione corretta. Al termine di ogni esecuzione del ciclo _for_, i primi _i_ elementi saranno ordinati e quando termina l'algoritmo tutto l'array sarà ordinato correttamente.

### Counting Sort

In [27]:
def counting_sort(arr):
    """Counting sort algorithm.

    Args:
        arr (list): Array to be sorted.

    Returns:
        list: Sorted array.

    Time complexity: O(n+k)

    k is the value of the maximum element in the array.
    """
    if not arr:
        return arr
    max_element = max(arr) #k
    count_array = [0] * (max_element + 1) #primo for

    for element in arr:
        count_array[element] += 1

    for i in range(1, len(count_array)):
        count_array[i] += count_array[i-1]

    sorted_array = [0] * len(arr)
    for element in reversed(arr):
        sorted_array[count_array[element]-1] = element #-1 perché indicizzazione parte da 0
        count_array[element] -= 1

    return sorted_array


Nella cella sovrastante troviamo l'algoritmo di ordinamento __Counting Sort__. Sottlineamo il fatto che questo algoritmi è utilizzabile solo per liste di numeri interi. Non è un algoritmo che opera per confronti, ma si basa sul determinare la posizione di ogni elemento nell'array ordinato in base a quanti sono gli elementi minori o uguali ad esso. In particolare, è necessario un array di appoggio di dimensione _max_element_, dove _max_element_ non è altro che il valore massimo presente nell'array di partenza. Successivamente, dopo aver inzializzato quest'array con tutti 0, incrementiamo di 1 l'elemento di indice _i_ per ogni elemento uguale a _i_ nell'array iniziale; poi scorriamo l'array di appoggio sommando ad ogni elemento tutti gli elementi ad esso precedenti. Quello che otteniamo dopo queste due operazioni non è altro che un array di appoggio in cui l'elemento di indice _x_ ha come valore il numero di elementi minori o uguali a _x_ nell'array iniziale. L'ultimo passaggio è quello di copiare su un nuovo array gli elementi ordinati, considerando che nell'array di appoggio l'elemento con indice _x_ risulta essere la posizione di _x_ nell'array ordinato. Decrementando poi progressivamente i valori dell'array di appoggio si riescono a gestire anche ventuali copie multiple di uno stesso valore.

## Implementazione e spiegazione dei test

In [28]:
import pandas as pd
import plotly.express as px
from enum import Enum
from dataclasses import dataclass, field
from timeit import default_timer as timer
import numpy as np


class InputType(Enum):
    """Select the type of input."""

    random = 1
    """Random input."""
    sorted = 2
    """Sorted input."""
    reversed = 3
    """Reversed input."""

class SelectTestType(Enum):
    """Select the type of sorting test."""

    insertion_sort = 1
    """Insertion sort."""
    counting_sort = 2
    """Counting sort."""

@dataclass
class InputConfig:
    """Input configuration."""

    num_samples: int = 1000
    """Number of samples."""
    sample_range: tuple[int, int] = (0, 5000)
    """Range of the samples."""
    input_type: InputType = InputType.random
    """Type of input."""


@dataclass
class InputGenerator:
    """Input generator."""

    input_config: InputConfig = InputConfig()
    """Input configuration."""

    data: list[int] = field(init=False, default_factory=list)
    """Data to be used for the tests."""

    def __post_init__(self):
        """Initialize the data."""
        self.data = self._generate()

    def _generate(self) -> list[int]:
        """Generate the data."""
        match self.input_config.input_type:
            case InputType.random:
                data = np.random.randint(
                    self.input_config.sample_range[0],
                    self.input_config.sample_range[1],
                    self.input_config.num_samples,
                )
            case InputType.sorted:
                data = np.sort(np.random.randint(
                    self.input_config.sample_range[0],
                    self.input_config.sample_range[1],
                    self.input_config.num_samples,
                ))
            case InputType.reversed:
                data = np.sort(np.random.randint(
                    self.input_config.sample_range[0],
                    self.input_config.sample_range[1],
                    self.input_config.num_samples,
                ))
                data = data[::-1]
            case _:
                raise ValueError("Invalid input type.")
        return data.tolist()

def insertion_test(input_data: list[int]) -> float:
    """Test the insertion sort algorithm."""
    start = timer()
    insertion_sort(input_data)
    end = timer()
    return end - start

def counting_test(input_data: list[int]) -> float:
    """Test the counting sort algorithm."""
    start = timer()
    counting_sort(input_data)
    end = timer()
    return end - start

def test(test_type: SelectTestType, input_type: InputType, num_samples: int, sample_range: tuple[int, int], step: int) -> pd.DataFrame:
    """Test the sorting algorithms.

    Args:
        test_type (SelectTestType): Type of sorting algorithm to test.
        input_type (InputType): Type of input.
        num_samples (int): Number of elements to sort.
        sample_range (tuple[int, int]): Range of the elements.
        step (int): Step to increase the number of elements.

        Returns:
            pd.DataFrame: Dataframe containing the results."""

    match test_type:
        case SelectTestType.insertion_sort:
            test_func = insertion_test
        case SelectTestType.counting_sort:
            test_func = counting_test
        case _:
            raise ValueError("Invalid queue type.")
    match input_type:
        case InputType.random:
            input_config = InputConfig(num_samples=num_samples, sample_range=sample_range, input_type=InputType.random)
        case InputType.sorted:
            input_config = InputConfig(num_samples=num_samples, sample_range=sample_range, input_type=InputType.sorted)
        case InputType.reversed:
            input_config = InputConfig(num_samples=num_samples, sample_range=sample_range, input_type=InputType.reversed)
        case _:
            raise ValueError("Invalid input type.")

    it = []
    for i in np.arange(0, num_samples, step):
        input_config.num_samples = i
        input_gen = InputGenerator(input_config=input_config)
        it.append(test_func(input_gen.data))

    times_df = pd.DataFrame(data={
        "num_samples": np.arange(0, num_samples, step),
        "time": it,
    })
    times_df["test_type"] = test_type.name
    times_df["input_type"] = input_type.name
    return times_df




Nella cella sovrastante sono state definite le funzioni e le classi necessarie per eseguire i test. In particolare, sono state definite le seguenti classi:
* _SelectTestType_: enum che contiene i tipi di test disponibili (insertion sort e counting sort)
* _InputType_: enum che contiene i tipi di input disponibili
* _InputConfig_: contiene i parametri necessari per generare l'input (numero di elementi, range degli elementi, tipo di input)
* _InputGenerator_: genera l'input in base ai parametri specificati

Inoltre, sono state definite le seguenti funzioni:
* _insertion_test_: esegue il test dell'algoritmo di insertion sort
* _counting_test_: esegue il test dell'algoritmo di counting sort
* _test_: esegue il test di uno dei due algoritmi di ordinamento, in base al tipo di test e di input scelti dall'utente. In particolare, questa funzione:
    * in base al tipo di test scelto, assegna alla variabile _test_func_ la funzione di test corrispondente
    * in base al tipo di input scelto, crea un oggetto _InputConfig_ con i parametri corrispondenti
    * i test vengono eseguiti su un numero di elementi che va da 0 a _num_samples_ con passo _step_
    * crea un oggetto _InputGenerator_ con l'oggetto _InputConfig_ creato precedentemente
    * esegue il test _test_func_ sull'input generato
    * crea un dataframe contenente i risultati del test e lo restituisce

    Il dataframe contiene le seguenti colonne:
        * _num_samples_: numero di elementi ordinati
        * _time_: tempo impiegato per ordinare i dati
        * _test_type_: tipo di test eseguito (insertion sort o counting sort)
        * _input_type_: tipo di input utilizzato (random, ordinato o inversamente ordinato)

In [29]:
def plot_results(num_samples: int, step: int, sample_range: tuple[int, int]):
    """Plot the results."""

    """Insertion sort tests"""

    df_ins_1 = test(SelectTestType.insertion_sort, InputType.random, num_samples, sample_range, step)
    df_ins_2 = test(SelectTestType.insertion_sort, InputType.sorted, num_samples, sample_range, step)
    df_ins_3 = test(SelectTestType.insertion_sort, InputType.reversed, num_samples, sample_range, step)

    """Counting sort tests"""

    df_cnt_1 = test(SelectTestType.counting_sort, InputType.random, num_samples, sample_range, step)
    df_cnt_2 = test(SelectTestType.counting_sort, InputType.sorted, num_samples, sample_range, step)
    df_cnt_3 = test(SelectTestType.counting_sort, InputType.reversed, num_samples, sample_range, step)

    df = pd.concat([df_ins_1, df_ins_2, df_ins_3, df_cnt_1, df_cnt_2, df_cnt_3], ignore_index=True)

    return df



Nella cella sovrastante è stata definita la funzione _plot_results_ che esegue tutti i test prendendo in input il numero di elementi dell'array, il passo con il quale devono essere eseguiti i test e il range di generazione dei numeri; restituisce un dataframe Pandas contenente i risultati dei test.
Nelle celle successive in cui viene eseguito codice, si mostrano i grafici voluti in base ai parametri scelti dall'utente.
In particolare, tramite il framework _plotly_ è possibile creare grafici interattivi che consentono di visualizzare i risultati dei test in maniera più chiara e comprensibile.

In [30]:
df = plot_results(1000, 10, (0, 100000))

figure = px.line(df.loc[df["num_samples"] > 0],
                 x="num_samples",
                 y="time",
                 color="test_type",
                 facet_col="input_type",
                 title="Sorting algorithms",
                 labels={"num_samples": "Dimensione array", "time": "Time (s)"})

figure.show()